# A Simple Neural Network Model

In [1]:
import numpy as np
import pandas as pd

# Input data files are available in the "./data/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('./data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

./data/test.csv
./data/train.csv
./data/gender_submission.csv


## Load the Data

### Training Data

In [2]:
train_data = pd.read_csv("./data/train.csv")
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### Test Data

In [3]:
test_data = pd.read_csv("./data/test.csv")
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## Preprocessing

In [4]:
feature_names = [
    "Pclass",
    "Sex",
    "Age",
    "SibSp",
    "Parch",
    "Fare",
    "Cabin",
    "Embarked"
]

target_name = ["Survived"]

X_train_orig = train_data[feature_names]
Y_train = train_data[target_name].to_numpy()

X_test_orig = test_data[feature_names]

### Handling Missing Values

Remove those features whose missing values account for more than 50%, since we consider features with more than this proportion of values being missing are not reliable to be used.

In [5]:
threshold = 0.5

# Concatenate `X_train_orig` and `X_test_orig` for now.
X_train_test = pd.concat([X_train_orig, X_test_orig], axis=0)
assert(len(X_train_test) == len(X_train_orig) + len(X_test_orig))

features_to_drop = []

# Loop through `feature_names`.
for feature in feature_names:
    feature_data = X_train_test[feature]
    
    # Find the number of missing values.
    num_missing_values = sum(pd.isna(feature_data))
    
    if num_missing_values / len(feature_data) >= threshold:
        features_to_drop.append(feature)

# Drop features.
print(features_to_drop)
X_train = X_train_orig.drop(features_to_drop, axis=1)
X_test = X_test_orig.drop(features_to_drop, axis=1)

X_train.head()

['Cabin']


,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,3,male,22.0,1,0,7.2500,S
1,1,female,38.0,1,0,71.2833,C
2,3,female,26.0,0,0,7.9250,S
3,1,female,35.0,1,0,53.1000,S
4,3,male,35.0,0,0,8.0500,S


Check whether remaining features have missing values or not.

In [6]:
features_with_missing_values = []

for i in X_train.columns:
    if sum(pd.isna(X_train[i])) > 0:
        features_with_missing_values.append(i)
        
for i in X_test.columns:
    if sum(pd.isna(X_test[i])) > 0:
        if i not in features_with_missing_values:
            features_with_missing_values.append(i)
        
print(features_with_missing_values)

['Age', 'Embarked', 'Fare']


Since we have categorical features being missing, using medians to replace missing values would be a sensible while simple strategy.

In [7]:
for feature in features_with_missing_values:
    feature_train_data = X_train[feature]
    feature_test_data = X_test[feature]

    missing_train_idx = pd.isna(feature_train_data)
    missing_test_idx = pd.isna(feature_test_data)
    nonmissing_train_idx = ~ missing_train_idx

    nonmissing_train_data = feature_train_data[nonmissing_train_idx].to_numpy(copy=True)
    nonmissing_train_data.sort()
    median = nonmissing_train_data[len(nonmissing_train_data) // 2]
    
    missing_train_idx = X_train.index[missing_train_idx]
    missing_test_idx = X_test.index[missing_test_idx]
    
    for i in missing_train_idx:
        X_train.loc[i, feature] = median
    for i in missing_test_idx:
        X_test.loc[i, feature] = median

Check the existence of missing values again.

In [8]:
features_with_missing_values = []

for i in X_train.columns:
    if sum(pd.isna(X_train[i])) > 0:
        features_with_missing_values.append(i)
        
for i in X_test.columns:
    if sum(pd.isna(X_test[i])) > 0:
        if i not in features_with_missing_values:
            features_with_missing_values.append(i)
        
print(features_with_missing_values)

[]


### One-Hot Encoding

`Pclass`, `Sex` and `Embarked` need to be one-hot encoded.

In [9]:
from sklearn.preprocessing import OneHotEncoder


feature_names = X_train.columns
categorical_features = [
    "Pclass",
    "Sex",
    "Embarked"
]
continuous_features = list(set(feature_names) - set(categorical_features))

# Retrieve data of categorical features.
X_train_categorical = X_train[categorical_features]
X_test_categorical = X_test[categorical_features]

# One-Hot encode all categorical features.
enc = OneHotEncoder(sparse=False)

X_train_categorical_encoded = enc.fit_transform(X_train_categorical)
X_test_categorical_encoded = enc.transform(X_test_categorical)

# Retrieve data of continuous features.
X_train_continuous = X_train[continuous_features]
X_test_continous = X_test[continuous_features]

# Reconstruct `X_train` & `X_test`.
X_train = np.concatenate([X_train_categorical_encoded, X_train_continuous], axis=1)
X_test = np.concatenate([X_test_categorical_encoded, X_test_continous], axis=1)
assert(len(X_train) == len(Y_train))